# 安装相关包

In [82]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import random
import csv

# 爬虫部分
利用requests库爬取网页内容

In [83]:
url='http://casad.cas.cn/ysxx2017/ysmdyjj/qtysmd_124280/'

heads = {}
heads['User-Agent'] = 'Mozilla/5.0 ' \
                          '(Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 ' \
                          '(KHTML, like Gecko) Version/5.1 Safari/534.50'

a = requests.get(url,headers=heads)
a.encoding='utf-8'

利用soup库进行解析，观察可知所需内容在标签<div class=contestTest></div>中
内容有规律的排布，<a href='链接'>人名</a>
因此遍历提取即可

In [86]:
soup = BeautifulSoup(a.text,'lxml')
div = soup.find('div',class_='contentTest')
div

<div class="contentTest" id="zoom">
<dl id="allNameBar">
<dt>数学物理学部<em>（146人）</em></dt>
<dd>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090626_1854608.html" target="_blank">艾国祥</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792084.html" target="_blank">白以龙</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/201711/t20171129_4625036.html" target="_blank">蔡荣根</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792083.html" target="_blank">陈彪</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792082.html" target="_blank">陈和生</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792081.html" target="_blank">陈佳洱</a>
</span>
<span>
<a href="http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792080.html" target="_blank">陈建生</a>

In [87]:
name_url=[]
name_dict={}
for i in div.findAll('a'):
    name_url.append([i.text,i.get('href')])
    name_dict[i.text]=[i.get('href')]

提取学部名称，添加进name_dict中

In [88]:
for i in div.text.split():
    if len(i)>4:
        department = i.split('（')[0]
        print(i)
    else:
        name_dict[i].append(department)

数学物理学部（146人）
化学部（123人）
生命科学和医学学部（145人）
地学部（127人）
信息技术科学部（93人）
技术科学部（135人）
（姓名按音序排列）


In [22]:
name_dict

{'艾国祥': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090626_1854608.html',
  '数学物理学部'],
 '白以龙': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792084.html',
  '数学物理学部'],
 '蔡荣根': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/201711/t20171129_4625036.html',
  '数学物理学部'],
 '陈彪': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792083.html',
  '数学物理学部'],
 '陈和生': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792082.html',
  '数学物理学部'],
 '陈佳洱': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792081.html',
  '数学物理学部'],
 '陈建生': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792080.html',
  '数学物理学部'],
 '陈木法': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792079.html',
  '数学物理学部'],
 '陈难先': ['http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/sxwlxb/200906/t20090624_1792078.html',
  '数学物理学部'],
 '陈十一': ['http://casad.cas.cn/sourcedb

定义爬取每个人的介绍内容函数

In [ ]:
def get_content(url):
    response = requests.get(url=url,headers=heads)
    response.encoding='utf-8'
    soup=BeautifulSoup(response.text,'lxml')
    p=soup.find_all('div',class_='contentTest')
    
    templ=[]
    for i in p:
        templ.append(i.text)
    content='\n'.join(templ).strip().replace(u'\u3000',u'').replace(u'\xa0',u'')
    content= content.split('{}')[-1]
    
    return content

便利name_dict中的链接并将爬取内容记录于同一个字典中

In [89]:
for i in name_dict:
    if len(name_dict[i])<4:
        name_dict[i].append(get_content(name_dict[i][0]))
        time.sleep(random.randint(1,2))
        print(i)

艾国祥
白以龙
蔡荣根
陈彪
陈和生
陈佳洱
陈建生
陈木法
陈难先
陈十一
陈式刚
陈恕行
陈仙辉
陈永川
陈志明
崔向群
戴元本
邓小刚
杜江峰
鄂维南
范海福
方成
方复全
方守贤
冯端
甘子钊
高鸿钧
葛墨林
龚昌德
龚新高
郭柏灵
郭尚平
韩占文
何国威
何祚庥
贺贤土
洪家兴
胡和生
胡仁宇
霍裕平
江松
姜伯驹
解思深
景益鹏
邝宇平
李安民
李邦河
李大潜
李德平
李方华
李家春
李家明
李儒新
李惕碚
励建书
林群
龙以明
罗俊
罗民兴
吕敏
马余刚
马志明
莫毅明
欧阳颀
欧阳钟灿
潘建伟
彭实戈
曲钦岳
沈文庆
沈学础
石钟慈
苏定强
苏肇冰
孙昌璞
孙鑫
孙义燧
汤涛
唐孝威
陶瑞宝
田刚
童秉纲
万哲先
汪承灏
汪景琇
王鼎盛
王恩哥
王广厚
王乃彦
王诗宬
王世绩
王绶琯
王小云
王迅
王贻芳
王元
王梓坤
魏宝文
文兰
吴岳良
武向平
席南华
夏道行
向涛
谢心澄
邢定钰
熊大闰
徐红星
徐叙瑢
徐至展
严加安
杨福家
杨国桢
杨乐
杨应昌
杨振宁
叶朝辉
叶叔华
于渌
袁亚湘
詹文龙
张殿琳
张恭庆
张涵信
张焕乔
张杰
张平文
张仁和
张淑仪
张维岩
张伟平
张裕恒
张肇西
张宗烨
赵光达
赵政国
赵忠贤
郑厚植
郑晓静
周光召
周恒
周向宇
周又元
周毓麟
朱邦芬
朱诗尧
邹广田
安立佳
白春礼
包信和
曹镛
柴之芳
陈洪渊
陈军
陈俊武
陈凯先
陈庆云
陈小明
陈新滋
陈懿
程津培
程镕时
戴立信
丁奎岭
段雪
方维海
费维扬
冯守华
冯小明
高松
郭景坤
郭子建
韩布兴
何国钟
何鸣元
洪茂椿
侯建国
胡英
黄本立
黄春辉
黄乃正
计亮年
江桂斌
江雷
江龙
江明
黎乐民
李灿
李洪钟
李静海
李亚栋
李永舫
李玉良
林国强
刘若庄
刘元方
刘云圻
刘忠范
陆熙炎
麻生明
麦松威
倪嘉缵
彭孝军
钱逸泰
任詠华
沙国河
沈家骢
沈之荃
宋礼成
孙世刚
谭蔚泓
唐本忠
唐勇
唐有祺
田禾
田昭武
田中群
佟振合
涂永强
万惠霖
万立骏
汪尔康
王方定
王佛松
王夔
吴奇
吴新涛
吴养洁
吴云东
席振峰
谢毅
谢毓元
谢在库
谢作伟
徐如人
严纯华
颜德岳
杨万泰
杨秀荣
杨学明
杨玉良
姚建年
姚守拙
于吉红
余国琮
俞汝勤
袁权
岳建民
张存浩
张东辉
张洪杰
张礼和
张俐娜
张乾二
张锁江
张涛
张希
张玉奎
赵东元
赵进才

修整格式并将爬取内容写入csv，方便进一步处理

（否则程序关了就白爬了）

In [108]:
for i in name_dict:
    name_dict[i][-1]=name_dict[i][-1].replace('\n',u'\xa0')
name_dict['王元'][-1]='数学家 1930年4月生于浙江兰溪，籍贯江苏镇江。1952年毕业于浙江大学。1980年当选为中国科学院学部委员(院士)。中国科学院数学与系统科学研究院研究员。曾任中国科学院数学研究所所长。 主要从事解析数论研究。20世纪50年代至60年代初，首先在中国将筛法用于哥德巴赫猜想研究，并证明了命题{3,4}，1957年又证明{2,3}，这是中国学者首次在此研究领域跃居世界领先地位。1973年与华罗庚合作证明用分圆域的独立单位系构造高维单位立方体的一致分布点贯的一般定理，被国际学术界称为“华－王方法”。70年代后期对数论在近似分析中的应用作了系统总结。80年代在丢番图分析方面，将施密特定理推广到任何代数数域，在丢番图不等式组等方面作出先进的工作。 1982年获国家自然科学奖一等奖，1990年获陈嘉庚物质科学奖。 '

In [ ]:
import csv
l=[]
for i in name_dict:
    l.append([i]+name_dict[i])
with open('yuanshi.csv','w',encoding='utf-8')as f:
    f_csv = csv.writer(f)
    f_csv.writerows(l)

# 数据清洗
写一个读取函数，以后进一步处理运行下面的部分就可

In [94]:
with open('yuanshi.csv','r',encoding='utf-8')as f:
    csv_reader = csv.reader(f)  # 使用csv.reader读取csvfile中的文件
    licsv=list(csv_reader)
name_dict={}
for i in licsv:
    if i!=[]:
        name_dict[i[0]]=i[1:]

建立新的字典dict_bsc，形如{'杨振宁':{'姓名':'杨振宁','学部':''},'刘丛强':{} }

In [95]:
dict_bsc={}
for i in name_dict:
    dict_bsc[i]={}
    dict_bsc[i]['姓名']=i
    dict_bsc[i]['学部']=name_dict[i][1]
    dict_bsc[i]['链接']=name_dict[i][0]
    dict_bsc[i]['网页内容']=name_dict[i][2]
    for j in ['本科学位','生于','籍贯','硕士学位','博士学位','博士后',"中国科学院院士",'第三世界科学院院士','中国科学院院士当选年份','师从','教授']:
        dict_bsc[i][j]=''

用正则表达式将句子分开，并根据句子的不同内容提取相关句子，待进一步提取所需数据


In [96]:
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s",name_dict[i][-1])
    
    # 处理 本科学位
    ## 观察到有的分的太开，需要前面的学校专业年份信息，所以合并
    for j in range(len(li)):
        if li[j] in ['获学士学位' ,'获理学学士学位', '获理学士学位','获工学士学位' ,'先后获学士', '博士学位','获该校学士学位与硕士学位']:
            li[j-1]+=li[j]
            li[j]=''
            #print(i,li)
    li = list(filter(None,li))
    c = 0
    for j in range(len(li)):
        if '本科' in li[j] or '学士' in li[j] or '就读于' in li[j]: #  '毕业于' in li[j] or
            c+=1
            if c>1: 
                print(i,li)
            else:
                dict_bsc[i]['本科学位']=li[j]
    for j in range(len(li)):
        if '毕业于' in li[j]:
            if dict_bsc[i]['本科学位']=='':
                dict_bsc[i]['本科学位']=li[j]
                
                
    # 处理 出生
    for j in range(len(li)):
        if '生于' in li[j]:
            dict_bsc[i]['生于']=li[j]
            
            
    # 处理 籍贯
    c=0
    for j in range(len(li)):
        if '原籍' in li[j] or '籍贯' in li[j]:
            c+=1
            if c>1: print(i,li)
            dict_bsc[i]['籍贯']=li[j][2:] # 前两个字直接去掉
            
    # 处理 硕士
    c=0
    for j in range(len(li)):
        if '硕士' in li[j] :
            c+=1
            if c>1: 
                # print(i,li)
                dict_bsc[i]['硕士学位']+='$'+li[j]
            else:
                dict_bsc[i]['硕士学位']=li[j]      
                
    # 处理 博士
    c=0
    for j in range(len(li)):
        if '博士' in li[j] and '博士后' not in li[j] :
            #print(i,li)
            c+=1
            if c>1: 
                #print(i,li)
                dict_bsc[i]['博士学位']+='$'+li[j]
            else:
                dict_bsc[i]['博士学位']=li[j]
                
    # 处理 博士后
    c=0
    for j in range(len(li)):
        if '博士后' in li[j] :
            c+=1
            if c>1: 
                #print(i,li)
                dict_bsc[i]['博士后']+='$'+li[j]
            else:
                dict_bsc[i]['博士后']=li[j]
                
    # 处理 中国科学院院士
    c=0
    for j in range(len(li)):
        if '中国科学院院士' in li[j] or '学部委员' in li[j] :
            c+=1
            dict_bsc[i]['中国科学院院士']=li[j]
            
    # 处理 第三世界科学院院士
    c=0
    for j in range(len(li)):
        if '第三世界科学院' in li[j] and '院士' in li[j]:
            c+=1
            dict_bsc[i]['第三世界科学院院士']=li[j]    
            
    # 处理 师从
    c=0
    for j in range(len(li)):
        if '师从' in li[j] :
            c+=1
            if c>1: 
                dict_bsc[i]['师从']+='$'+li[j]
            else:
                dict_bsc[i]['师从']=li[j]
                
    # 处理 教授
    c=0
    for j in range(len(li)):
        if '教授' in li[j] :
            c+=1
            if c>1: 
                dict_bsc[i]['教授']+='$'+li[j]
            else:
                dict_bsc[i]['教授']=li[j]

杨振宁 ['物理学家', '1922年9月出生于安徽合肥', '1938至1944年在国立西南联合大学物理系读书', '先后获学士、硕士学位', '1948年获美国芝加哥大学哲学博士学位', '清华大学高等研究院名誉院长、教授', '香港中文大学博文讲座教授', '20世纪五六十年代先后创立“杨－米尔斯规范场”论（Yang-Mills', 'gauge', 'theory）和提出“杨－巴克斯特方程”', '因与李政道共同提出弱相互作用中宇称不守恒原理而获1957年诺贝尔物理学奖', '历任普林斯顿高级研究所教授、纽约州立大学石溪分校爱因斯坦讲座教授兼理论物理研究所所长、香港中文大学博文讲座教授、洛克菲勒大学董事', '是美国国家科学院外籍院士', '美国物理学会以及巴西科学院、委内瑞拉科学院、西班牙皇家科学院、台北中央研究院院士', '英国皇家学会外籍会员、俄罗斯国家科学院外籍院士、日本科学院荣誉院士', '曾获美国“国家科学奖章”、美国费城富兰克林研究所的鲍威尔科学成就奖、费萨尔国王国际奖的科学奖', '著有《杨振宁论文选集》、《杨振宁文集》、《曙光集》等', '发表论文约300篇']
刘丛强 ['地球化学家', '1955年9月11日生于贵州省遵义市', '1982年2月于南京大学地球科学系毕业获理学学士学位', '1984年12月于中国科学院地球化学研究所获理学硕士学位', '后留所工作', '1986年5月留学日本', '先后在日本理化学研究所做访问学者', '日本东京大学理学部化学系攻读博士学位', '日本科学技术厅理化学研究所基础科学特别研究员', '日本国立电气通信大学化学系副教授', '1996年10月辞去副教授职务回国', '2011年当选为中国科学院院士', '2017年当选爱丁堡皇家学会（Royal', 'Society', 'of', 'Edinburgh）外籍院士', '中国科学院地球化学研究所研究员', '国家自然科学基金委员会副主任', '国家自然科学基金委副主任', '兼任中国矿物岩石地球化学学会名誉理事长', '曾任中科院地球化学研究所所长', '中国矿物岩石地球化学学会理事长', '主要从事（1）地表地球化学过程及其生态环境效应和（2）微量元素和同位素地球化学基础理论和应用两个方面的系统研究', '发表国际SCI收录论文170余篇

In [97]:
dict_bsc[i].keys()

dict_keys(['姓名', '学部', '链接', '网页内容', '本科学位', '生于', '籍贯', '硕士学位', '博士学位', '博士后', '中国科学院院士', '第三世界科学院院士', '中国科学院院士当选年份', '师从', '教授'])

对每一个进行细节处理

In [31]:
attr='第三世界科学院院士'
for i in dict_bsc:
    if dict_bsc[i][attr] !='':
        if re.search('\d{4}',dict_bsc[i][attr]) is not None:
            a,b=re.search('\d{4}',dict_bsc[i][attr]).span()
            dict_bsc[i]['第三世界科学院院士当选年份']=dict_bsc[i][attr][a:b]
        else:
            dict_bsc[i]['第三世界科学院院士当选年份']='未知'
attr='中国科学院院士'
for i in dict_bsc:
    if dict_bsc[i][attr] !='':
        if re.search('\d{4}',dict_bsc[i][attr]) is not None:
            a,b=re.search('\d{4}',dict_bsc[i][attr]).span()
            dict_bsc[i]['中国科学院院士当选年份']=dict_bsc[i][attr][a:b]
        else:
            dict_bsc[i]['中国科学院院士当选年份']='未知'

In [32]:
attr='生于'
dict_bsc['杨文采']['生于']='1942年10月3日生于广东大埔'
for i in dict_bsc:
    if dict_bsc[i][attr] !='':
        time,place=(re.split('出生于|生于',dict_bsc[i][attr]))
        if time!='':
            dict_bsc[i]['出生时间']=re.findall('\d{4}',time)[0]
        else: 
            dict_bsc[i]['出生时间']='未知'
        dict_bsc[i]['出生地点']=place

In [33]:
attr='本科学位'
dict_bsc['姚建年']['本科学位']='1982年毕业于福建师范大学化学系'
dict_bsc['段文晖']['本科学位']='1981-1992年在清华大学学习，先后于1986、1988与1992年获得该校学士、硕士与博士学位'

dict_bsc['杨振宁']['本科学位获得时间']='1942'
dict_bsc['杨振宁']['本科学位获得院系']='国立西南联合大学物理系'

dict_bsc['黄本立']['本科学位获得时间']='1949'
dict_bsc['黄本立']['本科学位获得院系']='岭南大学物理系'

# 段文晖 先后于1986、1988与1992年获得该校学士、硕士与博士学位 清华大学
dict_bsc['段文晖']['本科学位获得时间']='1986'
dict_bsc['段文晖']['本科学位获得院系']='清华大学'

dict_bsc['张统一']['本科学位获得时间']='未知'
dict_bsc['张统一']['本科学位获得院系']='河南师范大学'

dict_bsc['刘永坦']['本科学位获得时间']='未知' # 待check
dict_bsc['刘永坦']['本科学位获得院系']='哈尔滨工业大学'

dict_bsc['管晓宏']['本科学位获得时间']='1982' # 待check
dict_bsc['管晓宏']['本科学位获得院系']='清华大学'

university=['华中科技大学','南充师范学院（现西华师范大学）','浙江大学','南京大学地球科学系','兰州大学地质地理系','南京大学',
            '北京大学','国防科技大学','南京航空学院','北京理工大学','南京工业大学','西南联合大学','英国格拉斯哥大学']
u2=['华中科技大学','南充师范学院（现西华师范大学）','浙江大学','南京大学','兰州大学','南京大学',
            '北京大学','国防科技大学','南京航空学院','北京理工大学','南京工业大学','西南联合大学','英国格拉斯哥大学']
university2=[]
for i in dict_bsc:
    a = dict_bsc[i][attr] 
    if a !='':
        if '本科学位获得时间' not in dict_bsc[i].keys():
            if '毕业于' in a:
                time,place = a.split('毕业于')
                dict_bsc[i]['本科学位获得时间']=time[:4]
                dict_bsc[i]['本科学位获得院系']=place
                if '）' in place:
                    dict_bsc[i]['本科学位获得学校']=place.split('）')[0]+'）'
                else:
                    
                    if '大学' in place :
                        dict_bsc[i]['本科学位获得学校']=place.split('大学')[0]+'大学'
                    elif '学院' in place:
                        dict_bsc[i]['本科学位获得学校']=place.split('学院')[0]+'学院'
                   
                university2.append(place)
            else:
                print(i, a)
                dict_bsc[i]['本科学位获得时间']=a[:4]
                for j in university:
                    if j in a:
                        dict_bsc[i]['本科学位获得院系']=j
                for j in u2:
                    if j in a:
                        dict_bsc[i]['本科学位获得学校']=j
                        
for i in dict_bsc:
    a = dict_bsc[i]['本科学位'] 
university+=university2

郑晓静 1982年在华中科技大学获工学学士学位
魏辅文 1984年和1987年在南充师范学院（现西华师范大学）分别获得学士和硕士学位
丁仲礼 1982年在浙江大学获地球化学学士学位
刘丛强 1982年2月于南京大学地球科学系毕业获理学学士学位
姚檀栋 1978年在兰州大学地质地理系自然地理学专业本科毕业
张经 1982年在南京大学获学士学位
龚旗煌 1983年和1989年分别获得北京大学学士学位和博士学位（中英联合培养）
毛军发 1985年于国防科技大学获学士学位
梅宏 1984和1987年分别于南京航空学院获学士和硕士学位
吴一戎 1985、1988年先后在北京理工大学获学士和硕士学位
方岱宁 1982年和1986年分别在南京工业大学获本科和硕士学位
王希季 1942年西南联合大学学士
杨槱 1940年获英国格拉斯哥大学一等荣誉学士学位


In [34]:
university=list(set(university))
u=[]
for i in university:
    if '）' in i:
        u.append(i.split('）')[0]+'）')
    else:
        if '大学' in i :
            u.append(i.split('大学')[0]+'大学')
            # print(i.split('大学')[1])
        elif '学院' in i:
            u.append(i.split('学院')[0]+'学院')
            # print(i.split('学院')[1])

        else:
            u.append(i)
u+=u2
u=list(set(u))
u.append('中国科学院上海技术物理研究所')
u.append('中国科学院上海冶金研究所（现上海微系统与信息技术研究所）')

In [35]:
attr='硕士学位'


for i in dict_bsc:
    a = dict_bsc[i][attr] 
    if a!='':
        if '博士' not in a and '学士' not in a and '本科' not in a:
            # 处理时间
            if a[3] in [str(x) for x in range(10)]:
                dict_bsc[i]['硕士学位获得时间']=a[:4]
            else:
                None
                # print(i,a) # 手工处理
            # 处理院校
            for j in u:
                if j in a:
                    if '硕士学位获得院校' in dict_bsc[i].keys():
                        if(dict_bsc[i]['硕士学位获得院校']!=j):
                            None
                            #print('need check',i,dict_bsc[i]['硕士学位获得院校'],j)
                    else:
                        dict_bsc[i]['硕士学位获得院校']=j
            if '该校' in a:
                dict_bsc[i]['硕士学位获得院校']=dict_bsc[i]['本科学位获得学校']
            else:
                temp2 = re.sub('分别|先后|毕业|获|分|先|于|并|在|4月|1月|12月|\d{4}', "", re.sub('学理学硕士','学硕士',re.sub('学获理学硕士','学硕士',a))).split('年')[-1]
                temp2 = temp2.split('硕士')[0]
                dict_bsc[i]['硕士学位获得院校']=temp2
        else: 
            if '博士' not in a: # 学士硕士写在一句话
                temp=re.findall('\d{4}',a)
                if len(temp)==2:
                    dict_bsc[i]['硕士学位获得时间']=temp[1]
                    dict_bsc[i]['硕士学位获得院校']=dict_bsc[i]['本科学位获得学校']
                else:
                    None
                    # print(i,a) # 手工处理
            else: # 硕博写在一句话里
                temp=re.findall('\d{4}',a)
                if len(temp)==2:
                    dict_bsc[i]['硕士学位获得时间']=temp[0]
                    dict_bsc[i]['博士学位获得时间']=temp[1]
                    if '该校' in a:
                        dict_bsc[i]['硕士学位获得院校']=dict_bsc[i]['本科学位获得学校']
                        dict_bsc[i]['博士学位获得院校']=dict_bsc[i]['硕士学位获得院校']
                    else:
                        temp2=re.sub('分别|先后|获|分|先|于|在|6月|1月', "", a).split('年')[-1].split('硕士')[0]
                        if '协和' in temp2:
                            dict_bsc[i]['硕士学位获得院校']=temp2
                        else:
                            dict_bsc[i]['硕士学位获得院校']=temp2.split('和')[0]
                else:
                    None
                    # print(i,a) # 手工处理
# 手工输入
# 杨振宁 1938至1944年在国立西南联合大学物理系读书，先后获学士、硕士学位。
# 待check
dict_bsc['杨振宁']['硕士学位获得院校']='国立西南联合大学'
dict_bsc['杨振宁']['硕士学位获得时间']='1944' 
# 1985年至1988年在西安建筑科技大学读研究生，并获北京科技大学硕士学位，
# 北京科技大学何时存疑
dict_bsc['雒建斌']['硕士学位获得院校']='西安建筑科技大学'
dict_bsc['雒建斌']['硕士学位获得时间']='1988' 
# 吴良镛 1948-1950年在美国匡溪艺术学院建筑与城市设计系学习，并获硕士学位。
dict_bsc['吴良镛']['硕士学位获得院校']='美国匡溪艺术学院建筑与城市设计系'
dict_bsc['吴良镛']['硕士学位获得时间']='1950'
# 陈晔光 1986年获该校硕士学位$1990年获美国Fordham大学硕士学位
dict_bsc['陈晔光']['硕士学位获得时间2']='1990'
dict_bsc['陈晔光']['硕士学位获得院校2']='美国Fordham大学'
# 管晓宏 1985年毕业于清华大学获该校学士学位与硕士学位
dict_bsc['管晓宏']['硕士学位获得院校']='清华大学'
dict_bsc['管晓宏']['硕士学位获得时间']='1985'

# 叶嘉安 1976年获泰国亚洲理工学院硕士学位$1978和1980年分别获美国纽约州锡拉丘兹大学区域规划硕士及城市规划博士学位
dict_bsc['叶嘉安']['硕士学位获得院校']='泰国亚洲理工学院'
dict_bsc['叶嘉安']['硕士学位获得时间']='1976'
dict_bsc['叶嘉安']['硕士学位获得时间2']='1978'
dict_bsc['叶嘉安']['硕士学位获得院校2']='美国纽约州锡拉丘兹大学'
# 段文晖 先后于1986、1988与1992年获得该校学士、硕士与博士学位 清华大学
dict_bsc['段文晖']['硕士学位获得院校']='清华大学'
dict_bsc['段文晖']['硕士学位获得时间']='1988'
'''
翟明国 1982年和1989年在中国科学院研究生院和中国科学院地质研究所获理学硕士学位和理学博士学位
李树深 1989、1996年先后在西南交通大学和中国科学院半导体研究所获硕士和博士学位
程时杰 1981年和1986年分别在华中工学院和加拿大Calgary大学获硕士和博士学位
郑平 分别于1960年和1965年获美国麻省理工学院硕士学位和斯坦福大学工程博士学位
'''
dict_bsc['段文晖']['硕士学位获得院校']='中国科学院研究生院'
dict_bsc['李树深']['硕士学位获得院校']='西南交通大学'
dict_bsc['程时杰']['硕士学位获得院校']='华中工学院'
dict_bsc['郑平']['硕士学位获得院校']='美国麻省理工学院'
dict_bsc['段文晖']['博士学位获得院校']='中国科学院地质研究所'
dict_bsc['李树深']['博士学位获得院校']='中国科学院半导体研究所'
dict_bsc['程时杰']['博士学位获得院校']='加拿大Calgary大学'
dict_bsc['郑平']['博士学位获得院校']='斯坦福大学'
# 王成善 研究生成都地质学院（现成都理工大学）
dict_bsc['王成善']['硕士学位获得院校']='成都地质学院（现成都理工大学）'
# 黄荷凤 1989年获得妇产科学硕士学位
dict_bsc['黄荷凤']['硕士学位获得院校']=''
dict_bsc['黄荷凤']['硕士学位获得时间']='1989'

# 席振峰 1989年硕士研究生毕业于南京大学、郑州大学和河南化学研究所
dict_bsc['席振峰']['硕士学位获得院校']='!南京大学、郑州大学和河南化学研究所' # 待check
dict_bsc['席振峰']['硕士学位获得时间']='1989' # 待check

# 王恩多 1969年毕业于中国科学院上海生物化学研究所（1981年获硕士学位）
dict_bsc['王恩多']['硕士学位获得院校']='中国科学院上海生物化学研究所'
dict_bsc['王恩多']['硕士学位获得时间']='1981'


In [36]:
attr='博士学位'
'''
王梓坤 1958年获苏联莫斯科大学数学力学系副博士学位$1988年获澳大利亚麦克里大学名誉科学博士学位
张杰 1988年在中科院物理所获博士学位$是香港城市大学、英国女王大学、加拿大蒙特利尔大学、美国罗切斯特大学等大学的名誉博士
周向宇 1988年、1990年先后获中国科学院数学研究所硕士、博士学位$1998年获俄罗斯科学院Steklov数学所科学博士学位
黄乃正 1976年获英国伦敦大学大学学院哲学博士学位$1994年获英国伦敦大学科学博士学位
江雷 1992年至1994年日本东京大学中日联合培养博士$1994年回国后获吉林大学博士学位
汪尔康 1959年获捷克斯洛伐克科学院哲学博士学位$获全国百篇优秀博士论文的3人
陈晓亚 1985年获英国里丁大学博士学位$2010年获全国优秀博士学位论文指导教师
段树民 1991年获日本九洲大学博士学位$指导的学生多次获全国百篇优秀博士论文
魏江春 1962年获苏联科学院研究生院副博士学位$1995年获俄罗斯科学院科学博士学位
金之钧 1992年获俄罗斯莫斯科石油与天然气大学石油地质学副博士学位$2007年获俄罗斯联邦教育科学部地质矿物学博士学位
杨元喜 1991年获中国科学院测量与地球物理研究博士学位$曾任郑州测绘学院教授、博士导师
周卫健 1995年在西北大学地质系获博士学位$曾获全国首届百篇优秀博士学位论文
宋健 获副博士学位$后又获科学博士学位
吴一戎 中国科学院电子学研究所研究员、博士生导师$2001年在中国科学院电子学研究所获博士学位
姚期智 1972年在哈佛大学获物理学博士学位$1975年在伊利诺伊大学获计算机科学博士学位
何满潮 获博士学位$2011年获比利时MONS大学名誉博士
于起峰 1995年获德国不莱梅大学精密光测专业博士学位$现任国防科技大学航天科学与工程学院教授、博士生导师
张佑启 1964年获英国威尔斯大学博士学位$1973年获英国威尔斯大学科学博士学位$1982年获澳大利亚阿雷特大学工学博士学位
郑时龄 1993年获博士学位$2007年获意大利罗马大学名誉博士学位
'''
##费维扬 已培养了30余名博士生
# 洪茂椿 日本名古屋大学博士
# 侯建国 1978-1989年在中国科学技术大学学习，获博士学位
##张涛 博士
# 陈晔光 1996年获美国Albert Einstein 医学院博士学位
# 韩斌 1992年获英国John Innes Centre-The Sainsbury Laboratory博士学位
# 陆林 1999年毕业于华西医科大学, 获得医学博士学位
# 刘丛强 日本东京大学理学部化学系攻读博士学位
# 张经 1988于法国皮耶尔·玛丽居里大学（即：Paris 6）获博士学位
# 周秀骥 1962年苏联科学院应用地球物理研究所研究生毕业，获数理副博士学位
# 方岱宁 1993年在Technion-Isreal Institute of Technology获博士学位
# 韩祯祥 1961年毕业于前苏联莫斯科动力学院，获副博士学位。

##
# 谢作伟 1990年于该所获博士学位(与德国柏林工业大学联合培养)
# 王佛松 ##search: 1960年在苏联科学院列宁格勒高分子研究所研究生毕业并获副博士学位
# 
for i in dict_bsc:
    a = dict_bsc[i][attr] 
    if a!='':
        if '$' in a:
            # print(i,a)
            None
        else:
            if '硕士' not in a and '学士' not in a:
                if len(re.findall('\d{4}',a))!=1:
                    # print(i, a) # 单独处理
                    None
                else:
                    if '副博士' in a:
                        # print(i,a)
                        dict_bsc[i]['副博士学位获得时间']=re.findall('\d{4}',a)[0]
                        dict_bsc[i]['副博士学位获得院校']=re.sub('7月|获|毕业于|在|毕业|通过|“','',a).split('年')[-1].split('副博士')[0]
                    else:
                        if a[3] in [str(x) for x in range(10)]:
                            dict_bsc[i]['博士学位获得时间']=a[:4]
                        if '该' in a:
                            if dict_bsc[i]['硕士学位']!='':
                                dict_bsc[i]['博士学位获得院校']=dict_bsc[i]['硕士学位获得院校']
                            else:
                                dict_bsc[i]['博士学位获得院校']=dict_bsc[i]['本科学位获得学校']
                        elif '荣誉' in a or '名誉' in a:
                            None
                            # print(i,a)
                            #方成 2008年获法国巴黎天文台授予的荣誉博士学位
                            #田昭武 1984年获英国威尔士大学名誉博士学位
                            #张存浩 1998年获香港中文大学荣誉理学博士
                            #郭华东 2009年被澳大利亚科廷大学授予名誉科学博士学位
                            #黄宏嘉 1991年被授予名誉科学博士学位
                        else:
                            if '研究所' not in a and '研究院' not in a and '大学' not in a and '学院' not in a:
                                dict_bsc[i]['博士学位获得院校']=re.sub('分别|先后|获得|分|先|取得|于|在|毕业|\d{4}|4月|1月|获', "", re.sub('学理学博士','学博士',a)).split('年')[-1].split('博士')[0]
                            else:
                                # print(i,a)
                                a1=re.sub('分别|先后|获得|分|先|取得|于|在|毕业|\d{4}|4月|1月|获', "", re.sub('学理学博士','学博士',a)).split('年')[-1].split('博士')[0]
                                dict_bsc[i]['博士学位获得院校']=a1
dict_bsc['夏军']['博士学位获得院校']='武汉水利电力大学'
dict_bsc['秦大河']['博士学位获得院校']='兰州大学'
dict_bsc['唐有祺']['博士学位获得院校']='美国加州理工学院'
dict_bsc['杨文采']['博士学位获得院校']='加拿大McGill大学'
dict_bsc['胡聿贤']['博士学位获得院校']='美国密歇根大学'

In [37]:
attr='博士后'
for i in dict_bsc:
    a = dict_bsc[i][attr] 
    if a!='':
        print(i,a)

唐本忠 曾在加拿大多伦多大学从事博士后研究、日本NEOS公司中央研究所任高级研究员
唐有祺 并在该校任Hale博士后研究员
涂永强 曾在澳大利亚昆士兰大学作博士后、德国比勒费尔德大学作访问教授
杨玉良 1986～1988年在德国马普高分子研究所做博士后研究工作
周其林 先后在德国Max-Planck高分子研究所、瑞士Basel大学和美国Trinity大学从事博士后研究
舒德干 德国洪堡博士后
张培震 1987-1991在美国内华达大学从事博士后研究


In [46]:
# 国家自然科学奖
# 问题：有一些多年获奖的没统计所有时间
# 建议：扒国家自然科学奖库并格式化
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s",name_dict[i][-1])
    dict_bsc[i]['国家自然科学奖']=''
    dict_bsc[i]['国家自然科学奖获奖时间']=''
    for j in li:
        if '国家自然科学奖一' in j or '国家教育部自然科学奖' in j:
            dict_bsc[i]['国家自然科学奖']='$一等奖'
            if re.findall('\d{4}',j):
                dict_bsc[i]['国家自然科学奖获奖时间']+='$'+re.findall('\d{4}',j)[0]
        if '国家自然科学奖二' in j:
            dict_bsc[i]['国家自然科学奖']+='$二等奖'
            if re.findall('\d{4}',j):
                dict_bsc[i]['国家自然科学奖获奖时间']+='$'+re.findall('\d{4}',j)[0]
        if '国家自然科学奖三' in j :
            dict_bsc[i]['国家自然科学奖']+='$三等奖'
            if re.findall('\d{4}',j):
                dict_bsc[i]['国家自然科学奖获奖时间']+='$'+re.findall('\d{4}',j)[-1]
dict_bsc['滕吉文']['国家自然科学奖获奖时间']='$1987'
dict_bsc['张明杰']['国家自然科学奖获奖时间']='$2006'

In [47]:
# 国家科技进步奖
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s|、",name_dict[i][-1])
    dict_bsc[i]['国家科技进步奖']=''
    for j in li:
        if '国家科技进步奖特' in j:
            dict_bsc[i]['国家科技进步奖']+='$特等奖'
        if '国家科技进步奖一' in j:
            dict_bsc[i]['国家科技进步奖']+='$一等奖'
        if '国家科技进步奖二' in j:
            dict_bsc[i]['国家科技进步奖']+='$二等奖'
        if re.findall('\d{4}',j):
            dict_bsc[i]['国家科技进步奖获奖时间']=re.findall('\d{4}',j)[0]

In [48]:
# 军队科技进步奖
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s|、",name_dict[i][-1])
    dict_bsc[i]['军队科技进步奖']=''
    for j in li:
        if '军队科技进步奖' in j:
            if '军队科技进步奖一' in j:
                dict_bsc[i]['军队科技进步奖']+='$一等奖'
            else:
                dict_bsc[i]['军队科技进步奖']+='$二等奖'


In [49]:
# 国家技术发明奖 http://www.cas.cn/ky/kjjl/gjjsfmj/
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s|、",name_dict[i][-1])
    dict_bsc[i]['国家技术发明奖']=''
    for j in li:
        if '国家技术发明奖' in j or '国家发明奖' in j:
            if '国家技术发明奖一' in j:
                dict_bsc[i]['国家技术发明奖']+='$一等奖'
            else:
                dict_bsc[i]['国家技术发明奖']+='$二等奖'
dict_bsc['王崇愚']['国家技术发明奖']='国家发明奖'

In [50]:
#  其他各种奖项
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s|、",name_dict[i][-1])
    dict_bsc[i]['其他重要奖项']=''
    for j in li:
        for y in ['中国科学院杰出科技成就奖','全国科学大会奖','全国五一劳动奖章','全国科学大会成果奖']:
            if y in j :
                    dict_bsc[i]['其他重要奖项']+='$'+y
    

# 用其他信息来完善数据

In [38]:
with open('yuanshinianfen.csv','r',encoding='utf-8')as f:
    csv_reader = csv.reader(f)  # 使用csv.reader读取csvfile中的文件
    licsv=(list(csv_reader))
for i in licsv:
    for j in dict_bsc:
        if i[1]==j:
            dict_bsc[j]['性别']=i[-1]
            if dict_bsc[j]['中国科学院院士当选年份']!=i[-2] and dict_bsc[j]['中国科学院院士当选年份']!='':
                print('wrong',j,i[-2],dict_bsc[j]['中国科学院院士当选年份'])
                dict_bsc[j]['中国科学院院士当选年份']=i[-2]
            else:
                dict_bsc[j]['中国科学院院士当选年份']=i[-2]
            dict_bsc[j]['入选年龄'] = i[2]
            dict_bsc[j]['工作单位'] = i[3]
            dict_bsc[j]['专业'] = i[4]
'''
wrong 欧阳颀 2013 2015
wrong 张涛 2013 未知
wrong 金力 2013 2015
wrong 张培震 2013 2015
wrong 费维扬 2003 未知
'''

wrong 欧阳颀 2013 2015
wrong 张涛 2013 未知
wrong 金力 2013 2015
wrong 张培震 2013 2015
wrong 费维扬 2003 未知


'\nwrong 欧阳颀 2013 2015\nwrong 张涛 2013 未知\nwrong 金力 2013 2015\nwrong 张培震 2013 2015\nwrong 费维扬 2003 未知\n'

# 保存
将目前为止处理的全部结果保存至yuanshi_v1.csv

将所需要的结果保存至yuanshi_v2.csv

In [70]:
li=[]
for i in dict_bsc:
    li+=dict_bsc[i].keys()
li=list(set(li))
l_=[]
for i in dict_bsc:
    l__=[]
    for j in li:
        if j in dict_bsc[i].keys():
            l__.append(dict_bsc[i][j])
        else:
            l__.append('')
    l_.append(l__)

with open('yuanshi_v1.csv','w',encoding='utf-8',newline='')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(li)
    f_csv.writerows(l_)
lkey=sorted(li)

In [73]:
li=['姓名',
     '学部',
     '籍贯',
    '性别', 
    '工作单位', 
     '出生地点',
     '出生时间',
    '入选年龄',
     '本科学位获得时间',
     '本科学位获得学校',
     '硕士学位获得时间',
     '硕士学位获得院校',
     '硕士学位获得时间2',
     '硕士学位获得院校2',
     '副博士学位获得时间',
     '副博士学位获得院校',
     '博士学位获得时间',
     '博士学位获得院校',
     '中国科学院院士当选年份',
 '第三世界科学院院士当选年份',
    '国家自然科学奖', '国家自然科学奖获奖时间',
    '国家科技进步奖', '国家科技进步奖获奖时间', 
    '国家技术发明奖', 
   '军队科技进步奖',
    '其他重要奖项',
    '网页内容', '链接'
   ]
l_=[]
for i in dict_bsc:
    l__=[]
    for j in li:
        if j in dict_bsc[i].keys():
            l__.append(dict_bsc[i][j])
        else:
            l__.append('')
    l_.append(l__)
with open('yuanshi_v2.csv','w',encoding='utf-8',newline='')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(li)
    f_csv.writerows(l_)

利用pd在此查看内容

In [74]:
df=pd.DataFrame(l_,columns=li)

In [75]:
df

,姓名,学部,籍贯,性别,工作单位,出生地点,出生时间,入选年龄,本科学位获得时间,本科学位获得学校,...,第三世界科学院院士当选年份,国家自然科学奖,国家自然科学奖获奖时间,国家科技进步奖,国家科技进步奖获奖时间,国家技术发明奖,军队科技进步奖,其他重要奖项,网页内容,链接
0,艾国祥,数学物理学部,,,,湖南益阳,1938,,1963,北京大学,...,2002,,,$一等奖,1995,,,,天体物理学家，1938年2月生于湖南益阳。1963年毕业于北京大学地球物理系。1993年当选...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
1,白以龙,数学物理学部,浙江镇海,,,云南祥云,1940,,1963,中国科学技术大学,...,,$二等奖,$1993,,1993,,,,力学家 1940年12月生于云南祥云，籍贯浙江镇海。1963年毕业于中国科学技术大学力学系。...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
2,蔡荣根,数学物理学部,,男,引力理论和宇宙学,浙江省杭州市,1964,52,1985,杭州师范学院,...,,$二等奖,,,1100,,,,理论物理学家。中国科学院理论物理研究所研究员。1964年9月出生于浙江省杭州市。1985年毕...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
3,陈彪,数学物理学部,,,,,,,1946,成都金陵大学,...,,,,,1980,,,,天文学家 1923年11月出生，福建福州人。1946年毕业于成都金陵大学物理系。1980年当...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
4,陈和生,数学物理学部,福建省福州市,男,粒子物理,湖北省武汉市,1946,58,1970,北京大学,...,,,,,2005,,,,粒子物理学家 1946年生于湖北省武汉市，籍贯福建省福州市。1970年毕业于北京大学技术物理...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
5,陈佳洱,数学物理学部,,,,上海,1934,,1954,吉林大学,...,2001,,,,2001,,,,核物理学家 1934年10月出生于上海。1954年毕业于吉林大学物理系。1993年当选为中国...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
6,陈建生,数学物理学部,,,,福建福州人,1938,,1963,北京大学,...,,,,,1991,,,,天体物理学家。1938年7月生于福建福州人。1963年毕业于北京大学地球物理系。1991年当...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
7,陈木法,数学物理学部,,男,数学,福建惠安,1946,56岁,1969,北京师范大学,...,,,,,2003,,,,数学家 1946年8月生于福建惠安。1969年毕业于北京师范大学数学系，1980年该校研究生...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
8,陈难先,数学物理学部,浙江杭州,,,上海,1937,,1962,北京大学,...,,$二等奖,$1993,,1993,,,,物理学家 1937年10月生于上海，籍贯浙江杭州。1962年毕业于北京大学物理系。1984年...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...
9,陈十一,数学物理学部,,男,力学,浙江省天台县,1956,56,1981,浙江大学,...,,,,,2012,,,,力学家 1956年10月出生于浙江省天台县。1981年毕业于浙江大学力学系，1984年、19...,http://casad.cas.cn/sourcedb_ad_cas/zw2/ysxx/s...


# 用jieba分词分析一下还有什么词出现频率比较高的

In [53]:
import jieba
ljieba=[]
for i in dict_bsc:
    ljieba+=jieba.cut(dict_bsc[i]['网页内容'],cut_all=True)

In [101]:
djieba={}
for i in ljieba:
    if i not in djieba:
        djieba[i]=1
    else:
        djieba[i]+=1
# 按出现次数排序
so=sorted(djieba.items(),key=lambda x:x[1],reverse=True)
for i in so:
    if  '奖' in i[0]:
        print(i[0],i[1])
# print(so)
# 查看获得过什么奖
for i in name_dict:
    li=re.split(";|；|，|,|。|\.|\||\t|\s|、",name_dict[i][-1])
    for j in li:
        if '奖' in j:
            print(i,j)

二等奖 357
科学奖 263
一等奖 257
进步奖 188
国家自然科学奖 176
奖 167
国家科技进步奖 67
成就奖 60
奖励 57
科技奖 38
特等奖 36
发明奖 34
奖章 22
奖项 19
技术奖 18
成果奖 16
三等奖 15
贡献奖 15
物理奖 13
等奖项 12
金奖 10
化学奖 10
创新奖 9
国家发明奖 7
获奖 6
大奖 6
五一劳动奖章 5
集体奖 4
物理学奖 4
全国科学大会奖 4
国际奖 4
医学奖 4
全国五一劳动奖章 4
奖学 3
奖学金 3
金质奖 2
图书奖 2
最高奖 2
文学奖 1
荣誉奖 1
诺贝尔物理学奖 1
评奖 1
提名奖 1
诺贝尔化学奖 1
特别奖 1
优秀奖 1
奖状 1
金质奖章 1
政府奖 1
银质奖 1
银质奖章 1
个人奖 1
国家图书奖 1
图灵奖 1
大奖赛 1
二奖 1
四等奖 1
嘉奖 1
人居奖 1
艾国祥 1987年获国家科技进步奖一等奖
艾国祥 1994年获中国科学院自然科学奖一等奖
艾国祥 1995年获中国科学院科技进步奖一等奖
白以龙 1993年获国家自然科学奖二等奖
蔡荣根 曾获国家自然科学奖二等奖
蔡荣根 汤森路透全球高被引科学家奖
陈佳洱 获国家教委科技进步奖一等奖2项
陈佳洱 国家高技术研究发展计划“八五”先进工作者(一等奖)
陈难先 1993年获国家自然科学奖二等奖
陈恕行 曾获国家自然科学奖二等奖2项
陈仙辉 曾获长江学者成就奖
陈仙辉 求是杰出科技成就集体奖
陈仙辉 国家自然科学一等奖
陈仙辉 马蒂亚斯奖等
陈永川 曾获联合国教科文组织侯赛因青年科学家奖等
陈志明 曾获国家自然科学奖二等奖
陈志明 冯康科学计算奖
陈志明 中国数学会陈省身数学奖等
崔向群 曾获国家科技进步二等奖
崔向群 江苏省科技进步一等奖等
戴元本 1982年获国家自然科学奖二等奖
邓小刚 获军队科技进步一等奖2项
邓小刚 获首届军队科技创新群体奖及首届全国专业技术人才先进集体
杜江峰 曾获国家自然科学二等奖
杜江峰 教育部自然科学奖一等
杜江峰 中国物理学会黄昆物理奖等
鄂维南 奖
鄂维南 首届美国青年科学家和工程师总统奖
鄂维南 冯康科学计算奖等
范海福 并获得1987年国家自然科学二等奖
范海福 1991年中国物理学会叶企孙奖（第二获奖人）
范海福 1992年国家科技进步奖二

徐义刚 广东省科技进步奖一等奖
徐义刚 李四光地质科学奖等
许厚泽 获中国科学院科技进步奖一等奖2项
薛禹群 获江苏省科技成果奖一等奖
杨经绥 获国家自然科学奖二等奖
杨经绥 何梁何利科技进步奖和李四光奖等
杨树锋 曾获国家自然科学奖三等奖
杨树锋 教育部高等学校自然科学奖一等奖
杨树锋 浙江省科技进步奖
杨树锋 国家级教学成果奖等
杨元喜 1996获德国洪堡奖学金资助
杨元喜 2项成果获国家科技进步二等奖
杨元喜 5项成果获省部级科技进步一等奖
杨元喜 1999年获中国科协“求是杰出青年实用工程奖”
杨元喜 2011年获“何梁何利科技进步奖—地球科学奖”
姚檀栋 2017年获瑞典人类学和地理学会“维加奖”
叶嘉安 2008年获联合国人居署颁发的“联合国人居讲座奖”
殷鸿福 获国家自然科学奖二等奖2项
殷鸿福 湖北省自然科学奖一等奖
於崇文 1987年获地质矿产部科技成果奖一等奖
曾庆存 获国家自然科学奖二等奖2项和中国科学院自然科学奖一等奖5项
翟明国 获得国家自然科学二等奖二项
翟明国 中国科学院自然科学一等奖一项
翟明国 二等奖两项
翟明国 国土资源部科技创新一等奖一项
张国伟 1999年获国家自然科学奖二等奖
张宏福 曾获国家自然科学奖二等奖
张宏福 中国科学院杰出科技成就奖
张弥曼 1995年获国家自然科学奖二等奖
张培震 三次获国家科技进步奖二等奖
张培震 2005年获李四光地质科学奖
张人禾 曾获国家科技进步奖二等奖
张人禾 中国青年科技奖等学术奖励
赵柏林 1987年获国家科技进步奖一等奖
赵鹏大 获1992年国际数学地质协会最高奖——克伦宾奖章
郑度 1987年获国家自然科学奖一等奖
郑永飞 2004年获国家自然科学奖二等奖
郑永飞 2008年获何梁何利科技进步奖
郑永飞 2009年获长江学者成就奖
钟大赉 2001年两次获国家自然科学奖二等奖
周成虎 曾获国家科技进步奖二等奖等奖项
周卫健 国家自然科学二等奖
周卫健 三等奖
周志炎 1994年获中国科学院自然科学奖一等奖
周忠和 2003年获首届中科院杰出科学成就奖
周忠和 2007年获国家自然科学二等奖
朱日祥 1996年获中国科学院自然科学奖一等奖
邹才能 曾获国家科技进步奖一等奖
邹才能 二等奖
邹才能 李四光地质科学奖
陈定昌 曾获国家科技进步奖特等奖3项
陈定昌 2008年何梁何利科学